Our space embeddings recommender system is hosted as an API through the AI platform [Grace](https://2021.ai/offerings/grace-enterprise-ai-platform/) developed by 2021.ai. The platform lowers the barrier of entry for AI deployment, since there is no need for writing Docker and bash scripts.

Grace has plenty of well-written documentation available for all users, which made the deployment of our recommender system much simpler. If you are interested in a lightning quick guide to model deployment as an API, feel free to read the coming text. However, if you are simply interesting in seeing/making API calls you can scroll to the end of the document and find the code block. 

## Quick guide to model deployment in Grace

### Deployment folder setup
Since Grace makes use of [seldon](https://github.com/SeldonIO/seldon-core) for building the API, the folder structure of our deployment files has to follow a specific structure and contain specific files. This is how our deployment folder was structured:

```
deployment
|   definitions.py
|   deployment.yaml
|   Endpoint.py
|   requirements.txt
|
----serialized
    |   embeddings.pkl
    |   places_final.csv
```

The `definitions.py` always contains the same content as indicated below
```python
# definitions.yaml
import os
ROOT_DIR = os.path.dirname(os.path.abspath(__file__))
```

The `deployment.yaml` file also requires very low effort to fill out. The BaseImage is always set to the same in Grace. The ModelName is up to the modeller to use freely. Lastly, the ClassName should contain the name of the class which the API should call. It is <ins>very important</ins> that the class name is the same as the name of the file it is located in. In our project, it was chosen to use the default and named the class Endpoint and the python file `Endpoint.py` 
```yaml
BaseImage: seldonio/seldon-core-s2i-python3:1.2.3
ModelName: recommender-system
ClassName: Endpoint
```

The `Endpoint.py` file should have an initialization method `__init__` and a `predict` method. The seldon framework requires the predict method to take a ndarray (list or list of lists) and feature names as input, which can then be processed however you like.

```python
from definitions import ROOT_DIR
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle

class Endpoint(object):
    """
    Cutout of Endpoint class for explanation
    """

    def __init__(self):
        # Initializations 
        self.places = pd.read_csv(ROOT_DIR + "/serialized/places_final.csv")
        self.embeddings = pd.DataFrame(pickle.load(open(ROOT_DIR + "/serialized/continuous_embeddings.pkl", "rb"))).T

    def predict(self, x, feature_names):
        # Define user df
        user = pd.DataFrame(data=x, columns=feature_names)

        """
        Do something
        """

        return recommendation
```

The last file of the main folder, `requirements.txt`, contains the packages and versions to be used in the `Endpoint` class. In our Endpoint, we only use three packages as listed below:
```txt
numpy==1.18.1
pandas==1.0.1
scikit-learn==0.22.1
```

In the `serialized` folder, data and models are stored which can be used by the API. In our case, a data file `places.csv` and an embeddings file `embeddings.pkl`. You may have noticed that these files are loaded in the `__init__` method inside the Endpoint class.

In [1]:
import numpy as np
import requests

feature_names = ['IDs', 'Rating']
IDs = ['101742583391038750118','100574642292837870712']
Ratings = [4,2]

# Pythonic way to transpose list of lists 
data = list(map(list, zip(IDs, Ratings)))

In [2]:
# Construct data for endpoint
endpoint = "https://models.grace-dtu.2021.services/seldon/project-spaceembeddings/recommender-system/api/v0.1/predictions"
headers = {'Grace-Client-Secret': 'c0de6747-ffb6-4023-913f-53c8222435bb'}
payload = {"data": {"names": feature_names,
                    "ndarray": data}}

# Request response from endpoint
response = requests.post(endpoint, json=payload, headers=headers)

print(response.status_code)
print(response.json())

200
{'data': {'names': [], 'ndarray': ['NY145', 'NY4', 'NY243', 'NY2', 'NY245']}, 'meta': {}}
